<a href="https://colab.research.google.com/github/topel/RNN_pytorch_labs/blob/master/TP2_classification_de_noms_STUDENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#RNN-à-partir-de-caractères" data-toc-modified-id="RNN-à-partir-de-caractères-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RNN à partir de caractères</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Préparation des données</a></span></li><li><span><a href="#Création-du-RNN" data-toc-modified-id="Création-du-RNN-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Création du RNN</a></span></li><li><span><a href="#Création-de-votre-propre-modèle-RNN" data-toc-modified-id="Création-de-votre-propre-modèle-RNN-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Création de votre propre modèle RNN</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Évaluation" data-toc-modified-id="Évaluation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Évaluation</a></span></li><li><span><a href="#Visualizing-memorization" data-toc-modified-id="Visualizing-memorization-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Visualizing memorization</a></span></li></ul></div>

# RNN à partir de caractères

Dans ce TP, l'objectif est de construire un RNN simple qui essaye de prédire la langue d'origine d'un prénom/nom de famille. 

Ce TP montre comment faire le prétraitement de textes "à partir de zéro",sans utiliser les fonctions pratiques de torchtext.

Nous allons définir notre propre RNN en créant une classe RNN qui hérite de l'objet nn.Module de PyTorch. Nous prenons la prédiction finale du RNN comme étant la sortie finale, c'est-à-dire la classe à laquelle appartient le mot.

Nous allons entraîner le modèle sur quelques milliers de prénoms/noms de 18 langues d'origine, et prédire de quelle langue est un nouveau prénom/nom en fonction de l'orthographe.

Exemple :

nom = 'Dominique'<br/>
pred: 5 French<br/>
GT: 5 French<br/>


# Imports

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import random
import unicodedata
import string

import time
import math

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.autograd
import numpy as np

# Chargement des données

In [10]:
!wget -O dataset.zip https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/data_noms.zip
!ls -alth dataset.zip
!unzip -qq dataset.zip

--2019-12-09 17:10:55--  https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/data_noms.zip
Resolving www.irit.fr (www.irit.fr)... 141.115.28.2
Connecting to www.irit.fr (www.irit.fr)|141.115.28.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]   2.75M  1.68MB/s    in 1.6s    

2019-12-09 17:10:57 (1.68 MB/s) - ‘dataset.zip’ saved [2882130/2882130]

-rw-r--r-- 1 root root 2.8M Dec  9 14:06 dataset.zip
replace data/eng-fra.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Préparation des données

Le répertoire data/names contient 18 fichiers texte nommés "[Language].txt". Chaque fichier contient une liste de noms, un nom par ligne, le plus souvent romanisés (nous avons besoin de les convertir d'Unicode en ASCII).

Nous obtenons un dictionnaire de listes de noms par langue, {langue : [noms ...]}. Les variables génériques "category" et "line" (pour la langue et le nom dans notre cas) sont utilisées pour une extensibilité ultérieure possible.

In [11]:
def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Construction du dictionnaire category_lines, une liste de noms par langage
category_lines = {}
all_categories = []

# Lire un fichier et split en lignes
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/French.txt', 'data/names/Dutch.txt', 'data/names/Greek.txt', 'data/names/English.txt', 'data/names/Portuguese.txt', 'data/names/German.txt', 'data/names/Spanish.txt', 'data/names/Chinese.txt', 'data/names/Czech.txt', 'data/names/Vietnamese.txt', 'data/names/Korean.txt', 'data/names/Japanese.txt', 'data/names/Scottish.txt', 'data/names/Arabic.txt', 'data/names/Italian.txt', 'data/names/Polish.txt', 'data/names/Irish.txt', 'data/names/Russian.txt']
Slusarski


Maintenant nous avons category_lines, un dictionnaire mappant chaque catégorie (langue) à une liste de lignes (noms). Nous avons aussi gardé une trace de all_categories (une liste de langues) et de n_categories pour référence ultérieure.

In [12]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


Maintenant que nous avons tous les noms stockés, nous devons les transformer en tenseurs pour pouvoir les utiliser.

Pour représenter une lettre unique, nous utilisons un vecteur one-hot de taille <1 x n_letters>. 

Pour faire un mot, nous créons une matrice 2D <line_length x 1 x n_letters>.

Cette dimension supplémentaire est due au fait que PyTorch suppose que tout est en batchs - nous utilisons juste une taille de batch de 1 ici.

In [13]:
# Trouver l'indice d'une lettre dans all_letters, par exemple "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Pour transformer un Tensor <1 x n_letters>
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Transformer une ligne en un tenseur <line_length x 1 x n_letters>,
# ou un tableau de vecteurs one-hot
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print('J:', letterToTensor('J'))

print('Jones:', lineToTensor('Jones').size())

J: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
Jones: torch.Size([5, 1, 57])


# Création du RNN


Nous allons créer un module RNN (principalement copié du tutoriel PyTorch for Torch users), composé de 2 couches linéaires qui fonctionnent sur un état d'entrée et un état caché, avec une couche LogSoftmax en sortie.

Ce réseau est illustré dans la figure ci-dessous.

![rnn](https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/images/rnn_tp2.png)


In [14]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(??, ??)
        self.i2o = nn.Linear(??, ??)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((??, ??), 1)
        hidden = self.i2h(??)
        output = self.i2o(??)
        output = self.softmax(??)
        return ??, ??

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

SyntaxError: ignored

Pour exécuter une étape de ce réseau, nous devons passer une entrée (dans notre cas, le tenseur de la lettre courante) et l'état caché précédent (que nous initialisons d'abord sous forme d'un vecteur de zéros). Nous récupérons la sortie (probabilité de chaque langue) et un état caché suivant (que nous conserverons pour la prochaine étape).

In [0]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)
print(input.size(), hidden.size())
output, next_hidden = rnn(input, hidden)

Pour des raisons d'efficacité, nous ne voulons pas créer un nouveau Tensor pour chaque étape, donc nous allons utiliser lineToTensor au lieu de letterToTensor et utiliser des slices. Ceci pourrait être encore optimisé par le pré-calcul des batchs de tenseurs.

In [0]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

Ici le résultat est un Tensor <1 x n_categories>, où chaque élément est la probabilité de cette catégorie.



# Training

Définissons quelques fonctions utiles tout d'abord. La première sert à interpréter les résultats du réseau, qui sont les probabilités pour chaque catégorie. Nous utilisons Tensor.topk pour obtenir l'index de la plus grande valeur :

In [0]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

Nous voulons aussi un moyen rapide d'obtenir un exemple d'apprentissage (un nom et sa langue) :

In [0]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('categorie =', category, '/ ligne =', line)

Pour la fonction de perte nn.NLLLoss est appropriée, puisque la dernière couche du RNN est nn.LogSoftmax.

In [0]:
criterion = nn.NLLLoss()

In [0]:
learning_rate = 0.005 # Si trop grand, risque d'explosion. Si trop petit, risque de ne pas avoir d'apprentissage

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Update des params basique... SGD : ajout des gradients des parametres, multipliés par le learning rate 
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [0]:
n_iters = 100000
print_every = 5000
plot_every = 1000

# Liste des losses pour faire une figure
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Affichage de iter, loss, name et guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

In [0]:
plt.figure()
plt.plot(all_losses)
plt.show()

# Évaluation

In [0]:
# Liste des prédictions correctes dans une matrice de confusion
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Retourne la prédiction pour une ligne
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

# Passe sur un grand nb d'exemples et récupération du nombre de prédictions correctes
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normaliser en divisant chaque ligne par sa somme 
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Faire un plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# les axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# les ticks
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()


In [0]:
def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Obtenir les N top catégories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')

# Pour aller plus loin : "visualizing memorization"

In [0]:
# https://distill.pub/2019/memorization-in-rnns/

# nom = 'Dovesky'
# nom = 'Jackson'
# nom = 'Mohammed'
# nom = 'Sébastien'
nom = 'Dominique'
# nom='Thomas'

category_index = 5

line_tensor = lineToTensor(nom)
# print(line_tensor.size())
# print(line_tensor[0,0])
line_tensor.requires_grad_()

gradient_groundtruth_list = []
hidden = rnn.initHidden()
for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)
    g_groundtruth = torch.autograd.grad(output[0, category_index], line_tensor, retain_graph=True)[0].data
    gradient_groundtruth_list.append(g_groundtruth)
#     print(i, g.size(), g)
    
topv, topi = output.topk(1, 1, True)
category_index_predicted = topi[0][0].item()
print('pred: %i %s' %(category_index_predicted, all_categories[category_index_predicted]))
print('GT: %i %s' %(category_index, all_categories[category_index]))

gradient_pred_list = []
hidden = rnn.initHidden()
for i in range(line_tensor.size()[0]):
    output, hidden = rnn(line_tensor[i], hidden)
    g = torch.autograd.grad(output[0, category_index_predicted], line_tensor, retain_graph=True)[0].data
    gradient_pred_list.append(g)


# On calcule la "connectivity"
# https://discuss.pytorch.org/t/newbie-getting-the-gradient-with-respect-to-the-input/12709/2
# g = torch.autograd.grad(output[0,category_index], line_tensor, retain_graph=True)[0].data
# print(g.size())
# g = torch.autograd.grad(outputs[:,0,category_index], line_tensor, retain_graph=True)[0].data

connectivity = np.zeros((line_tensor.size()[0],line_tensor.size()[0]))

for i in range(line_tensor.size()[0]):
    char_index = letterToIndex(nom[i])
#     print(i, g[i,0,char_index])
    connectivity[:,i] = np.abs(gradient_groundtruth_list[i][:, 0, char_index].detach().cpu().numpy())
#     print(nom[i], connectivity[:,i])
    

plt.figure(figsize=(8,6))
plt.imshow(connectivity)
plt.ylabel("Time step", fontsize=14)
toto = plt.yticks(ticks=range(len(nom)), fontsize=14)
toto = plt.xticks(ticks=range(len(nom)), labels=list(nom), fontsize=14)

In [0]:
connectivity = np.zeros((line_tensor.size()[0],line_tensor.size()[0]))

for i in range(line_tensor.size()[0]):
    char_index = letterToIndex(nom[i])
#     print(i, g[i,0,char_index])
    connectivity[:,i] = np.abs(gradient_pred_list[i][:, 0, char_index].detach().cpu().numpy())
#     print(nom[i], connectivity[:,i])

plt.figure(figsize=(8,6))
plt.imshow(connectivity)
plt.ylabel("Time step", fontsize=14)
toto = plt.yticks(ticks=range(len(nom)), fontsize=14)
toto = plt.xticks(ticks=range(len(nom)), labels=list(nom), fontsize=14)

In [0]:
torch.autograd.grad?